# Mini Project

## Objectives

In [3]:
%load_ext sql
%config SqlMagic.displaylimit = None


displaylimit: Value None will be treated as 0 (no limit)

In [ ]:
##IGNORE
%sql sqlite:///go.db

Connecting to 'sqlite:///go.db'

: 

##IGNORE


```sqlite
(a)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
```
`CREATE INDEX`

# Project starts from here

In [ ]:
%%sql
-- Items table
CREATE TABLE Item (
    ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title TEXT NOT NULL,
    Type TEXT NOT NULL,
    Author TEXT,
    PublicationDate TEXT,  -- Format: YYYY-MM-DD
    Genre TEXT
);

-- Copies table
CREATE TABLE Copy (
    CopyID INTEGER PRIMARY KEY AUTOINCREMENT,
    ItemID INTEGER NOT NULL,
    Condition TEXT,
    Availability INTEGER NOT NULL DEFAULT 1,  -- 1: available, 0: not available
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

-- Users table
CREATE TABLE User (
    UserID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    ContactInfo TEXT,
    MembershipID TEXT UNIQUE,
    TotalFines REAL DEFAULT 0,
    IsVolunteer INTEGER DEFAULT 0  -- 0: no, 1: yes
);

-- Social Rooms table
CREATE TABLE SocialRoom (
    RoomID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Capacity INTEGER NOT NULL
);

-- Events table
CREATE TABLE Event (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Date TEXT NOT NULL,  -- YYYY-MM-DD
    Time TEXT NOT NULL,  -- HH:MM
    Description TEXT,
    RecommendedAudience TEXT,
    RoomID INTEGER NOT NULL,
    FOREIGN KEY (RoomID) REFERENCES SocialRoom(RoomID)
);

-- Personnel table
CREATE TABLE Personnel (
    PersonnelID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Position TEXT NOT NULL,
    ContactInfo TEXT
);

-- Future Items table
CREATE TABLE FutureItem (
    FutureItemID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title TEXT NOT NULL,
    Author TEXT,
    Type TEXT NOT NULL,
    ExpectedArrivalDate TEXT  -- YYYY-MM-DD
);

-- Borrows table
CREATE TABLE Borrows (
    UserID INTEGER,
    CopyID INTEGER,
    BorrowDate TEXT,  -- YYYY-MM-DD
    DueDate TEXT NOT NULL,
    ReturnDate TEXT,
    PRIMARY KEY (UserID, CopyID, BorrowDate),
    FOREIGN KEY (UserID) REFERENCES User(UserID),
    FOREIGN KEY (CopyID) REFERENCES Copy(CopyID)
);

-- Attends table
CREATE TABLE Attends (
    UserID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (UserID, EventID),
    FOREIGN KEY (UserID) REFERENCES User(UserID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

-- ManagesEvent table
CREATE TABLE ManagesEvent (
    PersonnelID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (PersonnelID, EventID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

-- ManagesItem table
CREATE TABLE ManagesItem (
    PersonnelID INTEGER,
    ItemID INTEGER,
    PRIMARY KEY (PersonnelID, ItemID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID),
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

-- RecommendedFor table
CREATE TABLE RecommendedFor (
    ItemID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (ItemID, EventID),
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

-- HelpRequest table
CREATE TABLE HelpRequest (
    RequestID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    PersonnelID INTEGER NOT NULL,
    RequestDate TEXT NOT NULL,  -- YYYY-MM-DD
    Issue TEXT NOT NULL,
    Status TEXT DEFAULT 'Pending',
    FOREIGN KEY (UserID) REFERENCES User(UserID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID)
);


CREATE TRIGGER update_availability_on_borrow
AFTER INSERT ON Borrows
WHEN NEW.ReturnDate IS NULL
FOR EACH ROW
BEGIN
    UPDATE Copy
    SET Availability = 0
    WHERE CopyID = NEW.CopyID;
END;




CREATE TRIGGER IF NOT EXISTS set_availability_on_return
AFTER UPDATE OF ReturnDate ON Borrows
WHEN NEW.ReturnDate IS NOT NULL AND OLD.ReturnDate IS NULL
BEGIN
UPDATE Copy
SET Availability = 1
WHERE CopyID = NEW.CopyID;
END;